In [1]:
#!pip install --upgrade gensim
import gensim
import json
import jsonpickle
from flask import Flask, request

import config
from gensim.summarization.bm25 import get_bm25_weights, BM25
import re

In [74]:
app = Flask(__name__)

@app.route('/ranking', methods=["POST"])
def ranging():
    documents = jsonpickle.decode(request.json)
    corpus = []
    for doc in documents["documents"]:

        text = re.split(" ", doc.text_normalized)
        corpus.append(text)
    
    query = re.split(" ", documents["query"])
    corpus.append(query)
    #print(corpus)
    
    bm25 = BM25(corpus)
    average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
    scores = bm25.get_scores(query, average_idf)
    
    sorted_corp = dict(zip(documents["documents"], scores[:-1]))
    print(sorted_corp)
    print("|"*50)
    ranked_docs = sorted(sorted_corp.items(), key=lambda kv: kv[1], reverse=True)
    
    return jsonpickle.encode(ranked_docs)


In [75]:
if __name__ == "__main__":

    #app.run(port=config.RANKING_PORT)
    app.run(host='0.0.0.0', port=config.RANKING_PORT)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13502/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Nov/2018 18:49:05] "POST /ranking HTTP/1.1" 200 -


{Title: Водитель погрузчика
Text: Режим работы:Полный рабочий деньЗарплата:400.00 руб.Вакансия:Водитель погрузчика (Погрузка вагонов г.Калинковичи.)Адрес:Гомельская область, г. Мозырь, , ул. Советская, 25Телефоны:8-0236-32-00-45, 32-05-98-юрист 8-0236-32-00-45, 8-0236-32-04-43


Snippet: None
: 0.41365832365201827, Title: Водитель автомобиля
Text: Образование:Профессионально-техническоеХарактер работы:ПостояннаяЗарплата:500.00 руб.Вакансия:Водитель автомобиля (вывозка леса)Адрес:Витебская область, г. Браслав, , ул. Дачная, 1А Телефоны:64585 8(02153) 64585 8(02153) 64585


Snippet: None
: 0.4377030486005748, Title: Водитель автомобиля
Text: Режим работы:Полный рабочий деньЗарплата:450.00 руб.Вакансия:Водитель автомобиля (категории "B", "C", "D")Адрес:г. Гродно, Ленинский район, , ул. Дзержинского, 104Телефоны:44-41-09 44-41-09


Snippet: None
: 0.4026001308415188, Title: Опытный водитель на своем aвто ( на фото ) к Вашим услугам и на Ваших условиях
Text: 73
 Надежный, профессиональный (